In [1]:
! pip install google-cloud-aiplatform
!pip install -q transformers peft accelerate bitsandbytes safetensors sentencepiece streamlit  langchain sentence-transformers tiktoken

!pip install pandas
!pip install llama-index
!pip install chromadb pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 35.4 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.7/174.7 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 111.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 92.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 11.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 92.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 109.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.2/174.2 kB 21.1 MB/s e

In [2]:
# # Automatically restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [2]:
from google.colab import auth as google_auth
google_auth.authenticate_user()

In [3]:
from langchain.llms import VertexAI

llmv = VertexAI(project="<>", endpoint_id="<>", max_output_tokens=2048)

In [4]:
from llama_index.llms import LangChainLLM
llm = LangChainLLM(llm=llmv)

In [5]:
import chromadb
db = chromadb.PersistentClient(path="./chroma_data")
chroma_collection = db.get_or_create_collection("quickstart")

In [6]:
from langchain.embeddings.huggingface import HuggingFaceBgeEmbeddings
embed_model = HuggingFaceBgeEmbeddings(model_name="BAAI/bge-base-en")

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/90.1k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/719 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader,ServiceContext
from llama_index.vector_stores import ChromaVectorStore
from llama_index.storage.storage_context import StorageContext

vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

service_context = ServiceContext.from_defaults(llm=llm, embed_model=embed_model)
documents = SimpleDirectoryReader('/content/drive/MyDrive/CMPE295/Prototype/data').load_data()
index = VectorStoreIndex.from_documents(documents, service_context=service_context,storage_context=storage_context )

[nltk_data] Downloading package punkt to /tmp/llama_index...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [10]:
from llama_index.memory import ChatMemoryBuffer

memory = ChatMemoryBuffer.from_defaults(token_limit=1500)

chat_engine = index.as_chat_engine(
    chat_mode="context",
    memory=memory,
)

In [11]:
response = chat_engine.stream_chat("What are the key principles of unix philosophy ? ")
for token in response.response_gen:
    print(token, end="")

 The key principles of the Unix philosophy are:

1. Small is beautiful: The Unix philosophy values small, simple programs that do one thing well. This makes the programs easier to understand, debug, and maintain.
2. Modularity: Programs should be modular, with well-defined interfaces that allow them to be combined to perform more complex tasks.
3. Reusability: Programs should be designed to be reusable, with the ability to be combined in new ways to solve different problems.
4. Use text streams as the universal interface: The Unix philosophy emphasizes the use of text streams as a universal interface between programs. This allows programs to be combined and manipulated easily.
5. Keep it simple and elegant: The Unix philosophy values simplicity and elegance in software design. Programs should be designed to be easy to understand and maintain, with a minimum of complexity.
6. Avoid reinventing the wheel: The Unix philosophy encourages the reuse of existing tools and libraries rather tha

In [13]:
response = chat_engine.stream_chat("What is the difference between flow routing and aggregation routing? explain in detail")
for token in response.response_gen:
    print(token, end="")

 Flow routing and aggregation routing are two different techniques used in computer networks to manage traffic flow. Flow routing is a technique that sets up individual flows in a network, while aggregation routing is a technique that groups together large numbers of flows with the same properties and sets up a single flow entry for the group.

Flow routing is typically used in networks where there is a need for fine-grained control over individual flows. For example, in a network where there are many different types of traffic, such as voice, video, and data, flow routing can be used to ensure that each type of traffic is given the appropriate priority and bandwidth. Flow routing can also be used to implement traffic engineering techniques, such as load balancing and congestion control.

Aggregation routing is typically used in networks where there are a large number of flows with the same properties. For example, in a network where there are many users who are all accessing the same 

In [ ]:
response = chat_engine.stream_chat("what is helm")
for token in response.response_gen:
    print(token, end="")

 Helm is a package manager for Kubernetes. It allows you to install, upgrade, and manage Kubernetes applications. Helm packages are called "charts" and they contain all of the resources that are needed to deploy an application to Kubernetes.
Helm is a popular tool for managing Kubernetes applications because it makes it easy to deploy and manage complex applications. Helm charts can be shared and reused, which makes it easy to deploy applications across multiple environments.
Here are some of the benefits of using Helm:
* Helm makes it easy to deploy and manage Kubernetes applications.
* Helm charts can be shared and reused, which makes it easy to deploy applications across multiple environments.
* Helm provides a consistent way to deploy and manage Kubernetes applications.
* Helm can be used to deploy applications to any Kubernetes cluster.

In [ ]:
chat_engine.reset()